In [2]:
# FB15k-test-rel.pkl
# FB15k-train-lhs.pkl
# FB15k-train-rhs.pkl
# FB15k_idx2entity.pkl
# FB15k_entity2idx.pkl

import cPickle as pickle
import numpy as np
import pprint
import ast

In [3]:
rhs = pickle.load(open('FB15k-train-rhs.pkl', 'r')).toarray()
lhs = pickle.load(open('FB15k-train-lhs.pkl', 'r')).toarray()
rel = pickle.load(open('FB15k-train-rel.pkl', 'r')).toarray()
entity2idx = pickle.load(open('../data/FB15k_entity2idx.pkl', 'r'))
idx2entity = pickle.load(open('../data/FB15k_idx2entity.pkl', 'r'))

data_path = '/Users/corbinrosset/Dropbox/Arora/QA-data/VanDurme_FB_annotations/annotated_clueweb/ClueWeb09_English_1/processed/'
execute_path = '/Users/corbinrosset/Dropbox/Arora/QA-code/src/process_clueweb/'
manifest_file = 'manifest.txt'

In [ ]:
print len(entity2idx.keys())
print len(idx2entity.keys())
print entity2idx

In [ ]:
import os, sys
import cPickle

import numpy as np
import scipy.sparse as sp

# Put the freebase15k data absolute path here
datapath = '/Users/corbinrosset/Desktop/QA_datasets/FB15k/'
assert datapath is not None

if 'data' not in os.listdir('../'):
    os.mkdir('../data')


def parseline(line):
    lhs, rel, rhs = line.split('\t')
    lhs = lhs.split(' ')
    rhs = rhs.split(' ')
    rel = rel.split(' ')
    return lhs, rel, rhs

#################################################
### Creation of the entities/indices dictionnaries

np.random.seed(753)

entities1 = set([])
entleftlist = []
entrightlist = []
rellist = []
relset1 = set([])

for datatyp in ['train']:
    f = open(datapath + 'freebase_mtr100_mte100-%s.txt' % datatyp, 'r')
    dat = f.readlines()
    f.close()
    for i in dat:
        lhs, rel, rhs = parseline(i[:-1])
        entleftlist += [lhs[0]]
        entrightlist += [rhs[0]]
        rellist += [rel[0]]
        
        rel = rel[0].replace('/', '.').strip('.')
        if '..' in rel:
            temp = rel.split('..')
            relset1.add(temp[0])
            relset1.add(temp[1])
        else:
            relset1.add(rel)
        
        entities1.add(lhs[0].replace('/', '.').strip('.'))
        entities1.add(rhs[0].replace('/', '.').strip('.'))

entleftset = np.sort(list(set(entleftlist) - set(entrightlist)))
entsharedset = np.sort(list(set(entleftlist) & set(entrightlist)))
entrightset = np.sort(list(set(entrightlist) - set(entleftlist)))
relset = np.sort(list(set(rellist)))



In [ ]:
# todo: filter clueweb triples to include only those available in FB15k...
###############################################################################
###         Global Settings and Paths  for Clueweb Data                     ###
###############################################################################

### relationship similarities:
print 'Comparing Relationships between FB15k and Clueweb Triples...'
rltn_freq = pickle.load(open(data_path + 'rltn_freq.pkl', 'r'))
relset2 =  set(rltn_freq.keys())
total_clueweb_triples = sum(rltn_freq.values())
print '\tTotal number of relationships (triples have many instances, and a given textual instance may express up to 2 relationships): ' + str(total_clueweb_triples)

overlap_rel = relset1 & relset2
print '\tFB15k relationship cardinality: ' + str(len(relset1))
print '\tClueweb relationship cardinality: ' + str(len(relset2))
sym_dif = relset1 ^ relset2
diff1 = relset2 - relset1
diff2 = relset1 - relset2
print '\tcommon relationships between FB15k and Clueweb: ' + str(len(overlap_rel))
print '\tNumber of textual instances expressing a common relationship: ' + str(sum([rltn_freq[i] for i in overlap_rel])) 
# pprint.pprint(diff2)


### entities
print 'Comparing Entities between FB15k and Clueweb...'
enty_freq = pickle.load(open(data_path + 'enty_id_freq.pkl', 'r'))
entities2 = set(enty_freq.keys())
total_entity_mentions = sum(enty_freq.values())
print '\tTotal number of entities mentioned (each triple has two entities): ' + str(total_entity_mentions)
print '\tTotal number of textual instances (half of the above): ' + str(total_entity_mentions/2)
print '\tCardinality of Clueweb entities: ' + str(len(entities2))
print '\tCardinality of FB15k entities: ' + str(len(entities1))

overlap_enty = entities1 & entities2
sym_dif = entities1 ^ entities2
diff1 = entities2 - entities1
diff2 = entities1 - entities2
print '\tcommon entities between FB15k and Clueweb: ' + str(len(overlap_enty))
print '\tNumber of textual instances expressing at least one common entity: ' + str(sum([enty_freq[i] for i in overlap_enty])) 
# pprint.pprint(diff2)


In [4]:


def process(formatted_data_chunk, chunk):
    for i, line in enumerate(formatted_data_chunk):
        ### format of each line:
        ### [relationships, reversd, (leftEntity, leftEntityID), result, (rightEntity, rightEntityID)]
        line = ast.literal_eval(line) ### convert text to data structure
        reversd = int(line[1])
        relationships = line[0]
        left = line[2]
        text = line[3] 
        right = line[4]
        
        leftID = '/' + left[1].replace('.', '/')
        rightID = '/' + right[1].replace('.', '/')
        relationship = '/' + './'.join([r.replace('.', '/') for r in relationships])
        
        triple = (leftID, relationship, rightID)
        
        if (leftID not in entities_set) or (rightID not in entities_set):
            print triple

    print 'all entities are valid'
        
#         print ''
#         if leftID not in entity2idx:
#             print str(i) + ' left not valid ' + str(triple)
#         if rightID not in entity2idx:
#             print str(i) + ' right not valid ' + str(triple)
#         if relationship not in entity2idx:
#             print str(i) + ' rel not valid ' + str(triple)
            
        
    
    
enty_freq = pickle.load(open(data_path + 'enty_id_freq.pkl', 'r'))
entities_set = set(enty_freq.keys())

### open manifest of formatted data
USE_FINAL = True
try:
    m = open(data_path + manifest_file, 'r')
    manifest_list = m.readlines()
    print 'loaded manifest of which data files need to be processed...'
except:
    print 'no manifest file, aborting'
    exit()

### collect statistics of the corpus chunk by chunk
for chunk in manifest_list[:1]:
    chunk = chunk.strip()
    if USE_FINAL: 
        chunk = chunk.replace('processed', 'final') ### use only final files
    try:
        formatted_data_chunk = open(data_path + chunk, 'r')
        print 'processing ' + str(chunk)
    except:
        print 'no \".processed\" file exists, run parse_text_anchors_vandurme.py on a corpus'
        continue
    process(formatted_data_chunk, chunk)

loaded manifest of which data files need to be processed...
processing en0000.final_0


AssertionError: 